In [1]:
import os
# save using vs code
SCALA_VERSION = '2.12'
SPARK_VERSION = '3.1.3'
# Download Kafka Jar file, this for readStream.format("kafka"), "kafka" is a driver
# kafka driver code is part of Maven Jar file
# https://mvnrepository.com/artifact/org.apache.spark/spark-sql-kafka-0-10_2.12/3.1.3
# pyspark-shell shall download the jar file behind..
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_{SCALA_VERSION}:{SPARK_VERSION} pyspark-shell'

In [2]:
# here we implement windowed candle data for 1 minute
# here we implement windowed candle data for 3 minute
# here we implement windowed candle data for 5 minute

# we use hadoop-vm as hostname, which is running on another system


#     kafka-topics  --create --bootstrap-server hadoop-vm:9092 --replication-factor 1 --partitions 1 --topic stock-ticks 
    
#     kafka-topics  --create --bootstrap-server hadoop-vm:9092 --replication-factor 1 --partitions 1 --topic candles-1min 
#     kafka-console-consumer --bootstrap-server hadoop-vm:9092 --topic  candles-1min  --from-beginning
 
#     kafka-topics  --create --bootstrap-server hadoop-vm:9092 --replication-factor 1 --partitions 1 --topic candles-3min 
#     kafka-console-consumer --bootstrap-server hadoop-vm:9092 --topic  candles-3min  --from-beginning

#     kafka-topics  --create --bootstrap-server hadoop-vm:9092 --replication-factor 1 --partitions 1 --topic candles-5min 
#     kafka-console-consumer --bootstrap-server hadoop-vm:9092 --topic  candles-5min  --from-beginning

In [3]:
import findspark
findspark.init()

In [4]:
import pyspark


from pyspark.sql import SparkSession
# spark groupBy has default setting for spark.sql.shuffle.partitions as 200
# we set to  4, should NOT be done in production 
spark = SparkSession.builder.master("local[1]")\
                            .config('spark.sql.shuffle.partitions', 4)\
                            .appName("SparkStreamingKafkaCandles").getOrCreate()

22/03/21 05:48:03 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/03/21 05:48:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark-3.1.3-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0a5cc83a-7f35-4dc4-bff6-b6a809aa84e6;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.3 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 572ms :: artifacts dl 11ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central

In [5]:
# read from kafka, here spark is consumer for kafka topic called test
# spark streaming works as dataframe/sql
# group.id is consumer group id
# subcribe is kafka topic
# "kafka" driver is not available by default with spark, we need to download it, we did on cell 1
# GROUP ID SHOULD BE CHANGED FOR EVERY PERSON 
kafkaDf = spark.readStream.format("kafka")\
  .option("kafka.bootstrap.servers", "hadoop-vm:9092")\
  .option("subscribe", "stock-ticks")\
  .option("group.id", "stock-ticks-group23rm3")\
  .load()

In [6]:
# key and value are binary type, we need to CAST To STRING type
kafkaDf.printSchema()
# timestampType values
# CreateTime:  Timestamp relates to message creation time as set by a Kafka client/producer
# LogAppendTime : Timestamp relates to the time a message was appended to a Kafka log.
# UnknownType

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [7]:
# Kafka value, key are in binary format
# we need to type cast to STRING, 
# we pick only value, timestamp as we need for 10 minute window
ticksDf = kafkaDf.selectExpr("CAST(value AS STRING)", "timestamp")
ticksDf.printSchema() # we get only value as string

root
 |-- value: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [8]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructField, StructType, DoubleType, StringType, LongType, TimestampType

schema = StructType( [
    StructField("symbol", StringType(), True),
    StructField("price", DoubleType(), True),
    StructField("volume", LongType(), True),
    StructField("timestamp", LongType(),  True)
])

In [9]:
# we parse value which is JSON string to kafka struct, 
# this create a new column [replacing existing value which is string] with type
jsonDf = ticksDf.withColumn("value", F.from_json("value", schema))
jsonDf.printSchema()

root
 |-- value: struct (nullable = true)
 |    |-- symbol: string (nullable = true)
 |    |-- price: double (nullable = true)
 |    |-- volume: long (nullable = true)
 |    |-- timestamp: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [10]:
# now extract all the sub fields symbol, price, volume, timestamp into data frame
# exisitng value, timestamp shall be removed. existing timestamp is value produced time, not a stock tick time
# value.timestamp that comes from producer is basically a event time
stockTickDf = jsonDf.select (F.col("value.*")) # extract all sub fields as schema
stockTickDf.printSchema()

root
 |-- symbol: string (nullable = true)
 |-- price: double (nullable = true)
 |-- volume: long (nullable = true)
 |-- timestamp: long (nullable = true)



In [11]:
# we have data coming in stream, each record has timestamp
# we have milli seconds 1647283268253, the actual seconds Mon Mar 14 2022 14:41:08
# now we have to calculate 1 minute window, sum(volume) up to 1 minute
# after conversions, we take timestamp which is truncatd to minute instead of milli seconds, drop temp columns
stockTickDf = stockTickDf\
                .withColumn("timestamp", (F.col("timestamp") / 1000).cast("timestamp"))\
                

stockTickDf.printSchema()
# echoOnconsole = stockTickDf\
#                 .writeStream\
#                 .outputMode("update")\
#                 .format("console")\
#                 .option("truncate", False)\
#                 .start() # start the query. spark will subscribe for data

root
 |-- symbol: string (nullable = true)
 |-- price: double (nullable = true)
 |-- volume: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [12]:
# aggregate
# sum (volume), sum(traded_value), candles open, high, low, close [OHLC]
# groupBy by symbol, timestamp

# now add delayed record processing with waterMark
stockTickDf1Min = stockTickDf\
                            .selectExpr("timestamp", "symbol",  "struct(*) as value")\
                            .withWatermark("timestamp", "3 minutes")\
                            .groupBy("symbol", F.window("timestamp", "180 seconds"))\
                          .agg( F.collect_list("value").alias("ticks"))

stockTickDf1Min.printSchema()

root
 |-- symbol: string (nullable = true)
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- ticks: array (nullable = false)
 |    |-- element: struct (containsNull = false)
 |    |    |-- symbol: string (nullable = true)
 |    |    |-- price: double (nullable = true)
 |    |    |-- volume: long (nullable = true)
 |    |    |-- timestamp: timestamp (nullable = true)



In [13]:
# spark stream with batch output
# spark basically process the data as batch
# for developer convinence, we have api, that can give us finite data frame for writing to file/jsbc/db/custom coding
# process discrete output batch
# this function is called for every update/delete/comlete triggers with result data set as dataframe
# candleBatchDf is BATCH DATAFRAME
# stockTickDf1Min is STREAM DATAFRAME
def processBatchData(candleBatchDf, batch_id):
    from pyspark.sql.functions import explode, col

    print ("process batch called", batch_id, "writing ", candleBatchDf.count())
    #result = candleBatchDf.collect()
    #print(result)
    #candleBatchDf.printSchema()
    #candleBatchDf.show(2, truncate = False)
    
    # pick all the elements inside ticks array into element
    # breaking group using explode
    candleTickBatchDf = (candleBatchDf.select("window", explode(F.column("ticks")) )
                                      .select(F.col("window.*"), F.col("col.*"))
                        
    
    # candleTickBatchDf.printSchema()
    # candleTickBatchDf.show(2, truncate=False)
    # write to csv/json/parqeut/database /jdbc etc
        .write
        .mode("append")
        .format("jdbc")
        .option("url", "jdbc:mysql://localhost:3306/stockdb?autoReconnect=true&useSSL=false&allowPublicKeyRetrieval=true")
        .option("driver", "com.mysql.jdbc.Driver")
        .option("user", "team")
        .option("password", "Team1234!")
        .option("dbtable", "candle_3min")
         .save())
    
    # window is nested column, has its children columns called start and end
    # start and end are part of nested window column
    # append mode will not delete existin data, instead append to existing table
     
    
stockTickDf1Min.writeStream.outputMode("append").foreachBatch(processBatchData).start()
    
# now go to mysql, check candle_1min exist
# show tables;
# select * from candle_1min

22/03/21 05:48:54 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-424884d7-3e44-4011-8f26-0d6b7b8d01d6. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


process batch called 0 writing  0


process batch called 1 writing  0
process batch called 2 writing  0
process batch called 3 writing  0
process batch called 4 writing  0


In [ ]:
# def processBatchData(candleBatchDf, batch_id):
#     from pyspark.sql.functions import explode, col

#     print ("process batch called", batch_id, "writing ", candleBatchDf.count())
#     #result = candleBatchDf.collect()
#     #print(result)
#     #candleBatchDf.printSchema()
#     #candleBatchDf.show(2, truncate = False)
    
#     # pick all the elements inside ticks array into element
#     # breaking group using explode
#     candleTickBatchDf = (candleBatchDf.select("window", explode(F.column("ticks")) )
#                                       .select(F.col("window.*"), F.col("col.*"))
#                                  .coalesce(1)
#                                  .write
#                                 .outputMode("append")
#                                 .format("csv")
#                                 .option("header", True)
#                                  .save( "hdfs://localhost:9000/dump3min")
#                             )
# stockTickDf1Min.writeStream.outputMode("append").foreachBatch(processBatchData).start()
                        

process batch called 65 writing  0
process batch called 66 writing  0
process batch called 67 writing  0
process batch called 68 writing  0
process batch called 69 writing  0
process batch called 70 writing  0


In [15]:
import pyspark.sql.functions as F
# output DataStreamWriter
# copy symbol to _symbol for partition, as spark shall remove the column 
# if we use  the column in partitionBy

(
    stockTickDf
     .withColumn("year", F.date_format(F.col("timestamp"), "yyyy"))
     .withColumn("month", F.date_format(F.col("timestamp"), "MM"))
     .withColumn("day", F.date_format(F.col("timestamp"), "dd"))  
     .withColumn("hour", F.date_format(F.col("timestamp"), "HH"))   
     .withColumn("_symbol", F.col("symbol"))   
    .writeStream
    .trigger(processingTime='65 seconds')\
    .queryName("Write Ticks to CSV trigger by 3min hour")
    .format("csv")
    .option("path", "hdfs://localhost:9000/dump-csv-trigger-hourly-3min")
    .option("header", True)
    .option("checkpointLocation", "hdfs://localhost:9000/checkpoint/tickscsvtohdfs63min")
    .partitionBy("year", "month", "day", "hour", "_symbol")
    .option("truncate", False)
    .start()
)

process batch called 15 writing  0
process batch called 16 writing  0


process batch called 17 writing  0
process batch called 18 writing  0


process batch called 19 writing  0
process batch called 20 writing  0
process batch called 21 writing  0
process batch called 22 writing  0
process batch called 23 writing  0
process batch called 24 writing  0
process batch called 25 writing  0
process batch called 26 writing  0
process batch called 27 writing  0
process batch called 28 writing  0


process batch called 29 writing  0
process batch called 30 writing  0


process batch called 31 writing  0
process batch called 32 writing  0
process batch called 33 writing  0
process batch called 34 writing  0
process batch called 35 writing  0
process batch called 36 writing  0
process batch called 37 writing  0
process batch called 38 writing  0
process batch called 39 writing  0
process batch called 40 writing  0
process batch called 41 writing  0
process batch called 42 writing  0


...

process batch called 2317 writing  0
process batch called 2318 writing  0
process batch called 2319 writing  0
process batch called 2320 writing  0
process batch called 2321 writing  0
process batch called 2322 writing  0
process batch called 2323 writing  0
process batch called 2324 writing  0


process batch called 2325 writing  0
process batch called 2326 writing  0
process batch called 2327 writing  0
process batch called 2328 writing  4


process batch called 2329 writing  0
process batch called 2330 writing  0
process batch called 2331 writing  0
process batch called 2332 writing  0
process batch called 2333 writing  0
process batch called 2334 writing  0
process batch called 2335 writing  0
process batch called 2336 writing  0


process batch called 2337 writing  0
process batch called 2338 writing  0


process batch called 2339 writing  0
process batch called 2340 writing  0
process batch called 2341 writing  0
process batch called 2342 writing  0
